In [77]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.metrics import accuracy_score

In [78]:
data = sns.load_dataset('titanic')

In [79]:
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [56]:
data.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [57]:
data.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [58]:
df = data[['pclass', 'sex', 'sibsp', 'parch', 'who', 'alone']].copy()

In [59]:
sex = {'male': 0, 'female': 1}
df['sex'] = df['sex'].map(sex)

In [60]:
who = {'man': 0, 'woman': 1, 'child': 2}
df['who'] = df['who'].map(who)

In [61]:
alone = {True: 1, False: 0}
df['alone'] = df['alone'].map(alone)

In [62]:
df.head()

,pclass,sex,sibsp,parch,who,alone
0,3,0,1,0,0,0
1,1,1,1,0,1,0
2,3,1,0,0,1,1
3,1,1,1,0,1,0
4,3,0,0,0,0,1


In [63]:
X = df.copy()
y = data['survived']

In [64]:
X.shape, y.shape

((891, 6), (891,))

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# 1. Filter methods

## 1.1. Chi-square

In [66]:
f_score = chi2(X_train, y_train)

In [67]:
f_score

(array([ 21.06820598, 136.54504395,   3.86163625,   8.03684342,
        150.29613388,   9.37645739]),
 array([4.43221494e-06, 1.51634551e-31, 4.94021275e-02, 4.58353401e-03,
        1.49359320e-34, 2.19789895e-03]))

In [68]:
p_values = pd.Series(f_score[1], index = X_train.columns)
p_values.sort_values(ascending = True, inplace = True)

In [69]:
p_values

who       1.493593e-34
sex       1.516346e-31
pclass    4.432215e-06
alone     2.197899e-03
parch     4.583534e-03
sibsp     4.940213e-02
dtype: float64

# 2. Wrapper methods

## 2.1. Step Forward Feature Selection

In [84]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.metrics import accuracy_score as acc

In [85]:
# Build RF classifier to use in feature selection
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

# Build step forward feature selection
sfs = sfs(clf,
          k_features=6,
          forward=True,
          floating=False,
          verbose=2,
          scoring='accuracy',
          cv=5)

# Perform SFFS
sfs = sfs.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    5.7s finished

[2019-11-26 16:26:54] Features: 1/6 -- score: 0.7922158353084334[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.7s finished

[2019-11-26 16:26:59] Features: 2/6 -- score: 0.8161305081892436[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    4.0s finished

[2019-11-26 16:27:03] Features: 3/6 -- score: 0.830096683857547[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 o

In [86]:
# Looking at the selected features
feat_cols = list(sfs.k_feature_idx_)
print(feat_cols)

[0, 1, 2, 3, 4, 5]


In [26]:
# Build full model with selected features
clf = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=4)
clf.fit(X_train[[X_train.columns[0], X_train.columns[2], X_train.columns[4]]], y_train)

y_train_pred = clf.predict(X_train[[X_train.columns[0], X_train.columns[2], X_train.columns[4]]])
print('Training accuracy on selected features: %.3f' % acc(y_train, y_train_pred))

y_test_pred = clf.predict(X_train[[X_train.columns[0], X_train.columns[2], X_train.columns[4]]])
print('Testing accuracy on selected features: %.3f' % acc(y_test, y_test_pred))

Training accuracy on selected features: 0.824


ValueError: Found input variables with inconsistent numbers of samples: [179, 712]